In [5]:
from subgrounds.subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
import pandas as pd
import requests as r
import sys
sys.path.append('../')
import defillama_utils as dfl
import optimism_pool_tvls as tvls
from datetime import timedelta
import time
import os

### Data Sources
- Messari Subgraphs: [https://thegraph.com/hosted-service/subgraph/messari/curve-finance-ethereum](https://thegraph.com/hosted-service/subgraph/messari/curve-finance-ethereum)

In [6]:
chain_list = ['arbitrum','aurora','avalanche','ethereum',\
        'fantom','gnosis','harmony','moonbeam','optimism','polygon']
trailing_num_days = 180
start_time = pd.to_datetime("today") - timedelta(days = trailing_num_days-1)
start_time = int( time.mktime(start_time.timetuple()) )

# def get_messari_sg_pool_snapshots(slug, chains, min_ts, max_ts):
get_curve = tvls.get_messari_sg_pool_snapshots('curve-finance',chain_list,start_time)


curve-finance
arbitrum
['id', 'timestamp', 'totalValueLockedUSD', 'dailyVolumeUSD', 'rewardTokenEmissionsUSD', 'protocol_id', 'protocol_name', 'protocol_slug', 'protocol_network', 'pool_id', 'pool_name', 'pool_symbol']


### Export and Show

In [7]:
if not os.path.exists('csv_outputs'):
    os.makedirs('csv_outputs')
get_curve.to_csv('csv_outputs/curve_snapshot_data.csv')
# display(get_curve)

In [8]:
# group by the specified columns
grouped = get_curve.sort_values(by='timestamp', ascending=False)\
                  .groupby(['protocol_network', 'pool_id', 'pool_name', 'pool_symbol'])

# perform the desired aggregations
aggregated = grouped.agg({'totalValueLockedUSD': 'first', 'dailyVolumeUSD': 'mean', 'timestamp':'nunique'})

aggregated = aggregated.rename(columns={'totalValueLockedUSD':'latest_tvl_usd'
                                        ,'dailyVolumeUSD':'avg_daily_volume_usd'
                                        ,'timestamp':'days_of_trading'})

# reset the index
aggregated = aggregated.reset_index()

aggregated = aggregated[aggregated['pool_id'] != 0]

display(aggregated.sort_values(by='days_of_trading',ascending=False))
aggregated.to_csv('csv_outputs/curve_aggregated_data.csv')

,protocol_network,pool_id,pool_name,pool_symbol,latest_tvl_usd,avg_daily_volume_usd,days_of_trading
20,ARBITRUM_ONE,0x960ea3e3c7fb317332d990873d354e18d7645590,Curve.fi USD-BTC-ETH,crv3crypto,2.480670e+07,4.524877e+06,180
19,ARBITRUM_ONE,0x7f90122bf0700f9e7e1f688fe926940e8839f353,Curve.fi USDC/USDT,2CRV,2.432677e+07,8.385104e+05,180
17,ARBITRUM_ONE,0x76b44e0cf9bd024dbed09e1785df295d59770138,Curve.fi Factory USD Metapool: deBridge-USDC,deUSDС+2P3CRV-f,2.296906e+05,9.944287e+03,180
26,ARBITRUM_ONE,0xc9b8a3fdecb9d5b218d02555a8baf332e5b740d5,Curve.fi Factory Plain Pool: FRAXBP,FRAXBP-f,2.155510e+06,3.286591e+05,180
12,ARBITRUM_ONE,0x59bf0545fca0e5ad48e13da269facd2e8c886ba4,Curve.fi Factory Plain Pool: VST+FRAX,VSTFRAX-f,4.659461e+06,1.720021e+05,180
7,ARBITRUM_ONE,0x30df229cefa463e991e29d42db0bae2e122b2ac7,Curve.fi Factory USD Metapool: MIM,MIM3CRV-f,1.045589e+07,2.665278e+05,180
27,ARBITRUM_ONE,0xd0dd5d76cf0fc06dabc48632735566dca241a35e,Curve.fi Factory USD Metapool: handle3pool,fxUSD_2CRV3CRV-f,3.129779e+05,2.167547e+03,176
1,ARBITRUM_ONE,0x0a824b5d4c96ea0ec46306efbd34bf88fe1277e0,Curve.fi Factory Pool: deBridge-ETH,deETH+ETH-f,0.000000e+00,4.134685e+03,174
9,ARBITRUM_ONE,0x3e01dd8a5e1fb3481f0f589056b428fc308af0fb,Curve.fi WBTC/renBTC,btcCRV,4.812444e+05,1.620412e+04,172
5,ARBITRUM_ONE,0x2ce5fd6f6f4a159987eac99ff5158b7b62189acf,Curve.fi Factory USD Metapool: dForce,USX2CRV3CRV-f,1.532336e+06,7.580014e+03,170
